In [1]:
class Vertex:
    def __init__(self):
        self._links = []

    @property
    def links(self):
        return self._links
    
class Link:
    def __init__(self, v1, v2):
        self._v1 = v1
        self._v2 = v2
        self._dist = 1

    @property
    def v1(self):
        return self._v1
    
    @property
    def v2(self):
        return self._v2
    
    @property
    def dist(self):
        return self._dist
    
    @dist.setter
    def dist(self, dist):
        self._dist = dist

    def __eq__(self, other):
        return (self.v1 == other.v1 and self.v2 == other.v2) or (self.v1 == other.v2 and self.v2 == other.v1)

class LinkedGraph:
    def __init__(self):
        self._links = []
        self._vertex = []

    def matr_sm(self):
        matr = [[float('inf')] * len(self._vertex) for i in range(len(self._vertex))]
        for i in range(len(self._vertex)):
            for j in range(len(self._vertex)):
                if i == j:
                    matr[i][j] = 0
                else:
                    for lnk in self._links:
                        if lnk.v1 == self._vertex[i] and lnk.v2 == self._vertex[j]:
                            matr[i][j] = matr[j][i] = lnk.dist
                            break
        return matr

    def add_vertex(self, v):
        if v not in self._vertex:
            self._vertex.append(v)

    def add_link(self, link):
        if all((map(lambda x: link != x, self._links))):
            self._links.append(link)
            self.add_vertex(link.v1)
            self.add_vertex(link.v2)
            link.v1._links.append(link)
            link.v2._links.append(link)

    def find_path(self, start_v, stop_v):
        N = len(self._vertex)  # число вершин в графе
        T = [float('inf')]*N   # последняя строка таблицы
        v = 0       # стартовая вершина (нумерация с нуля)
        S = {v}     # просмотренные вершины
        T[v] = 0    # нулевой вес для стартовой вершины
        M = [0]*N   # оптимальные связи между вершинами
        D = self.matr_sm()  # матрица смежности
        LS = []

        while v != -1:          # цикл, пока не просмотрим все вершины
            for j, dw in enumerate(D[v]):   # перебираем все связанные вершины с вершиной v
                #print(j, dw, D[v])
                if j not in S:           # если вершина еще не просмотрена
                    w = T[v] + dw
                    if w < T[j]:
                        T[j] = w
                        M[j] = v        # связываем вершину j с вершиной v
            v = self.arg_min(T, S)            # выбираем следующий узел с наименьшим весом
            if v >= 0:                    # выбрана очередная вершина
                S.add(v)                 # добавляем новую вершину в рассмотрение

        start = self._vertex.index(start_v)
        end = self._vertex.index(stop_v)
        P = [end]
        while end != start:
            end = M[P[-1]]
            P.append(end)

        try:
            l = list(map(lambda x: self._vertex[x].name, P))[::-1]
        except:
            l = list(map(lambda x: self._vertex[x], P))[::-1]

        try:
            for i in range(len(l) - 1):
                y = list(filter(lambda x: x.v1.name == l[i] and x.v2.name == l[i + 1] or x.v1.name == l[i + 1] and x.v2.name == l[i], self._links))[0]
                LS.append(y)
        except:
            for i in range(len(l) - 1):
                y = list(filter(lambda x: x.v1 == l[i] and x.v2 == l[i + 1] or x.v1 == l[i + 1] and x.v2 == l[i], self._links))[0]
                LS.append(y)
        try:
            l = list(map(int, l))
        except:
            l = list(map(str, l))
        return l, LS

    def arg_min(self, T, S):
        amin = -1
        m = float('inf')  # максимальное значение
        for i, t in enumerate(T):
            if t < m and i not in S:
                m = t
                amin = i
        return amin

class Station(Vertex):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name

class LinkMetro(Link):
    def __init__(self, v1, v2, dist):
        super().__init__(v1, v2)
        self.dist = dist


map_metro = LinkedGraph()
v1 = Station("Сретенский бульвар")
v2 = Station("Тургеневская")
v3 = Station("Чистые пруды")
v4 = Station("Лубянка")
v5 = Station("Кузнецкий мост")
v6 = Station("Китай-город 1")
v7 = Station("Китай-город 2")

map_metro.add_link(LinkMetro(v1, v2, 1))
map_metro.add_link(LinkMetro(v2, v3, 1))
map_metro.add_link(LinkMetro(v1, v3, 1))

map_metro.add_link(LinkMetro(v4, v5, 1))
map_metro.add_link(LinkMetro(v6, v7, 1))

map_metro.add_link(LinkMetro(v2, v7, 5))
map_metro.add_link(LinkMetro(v3, v4, 3))
map_metro.add_link(LinkMetro(v5, v6, 3))


path = map_metro.find_path(v1, v6)  # от сретенского бульвара до китай-город 1

map2 = LinkedGraph()
v1 = Vertex()
v2 = Vertex()
v3 = Vertex()
v4 = Vertex()
v5 = Vertex()
map2.add_link(Link(v1, v2))
map2.add_link(Link(v2, v3))
map2.add_link(Link(v2, v4))
map2.add_link(Link(v3, v4))
map2.add_link(Link(v4, v5))
assert len(map2._links) == 5, "неверное число связей в списке _links класса LinkedGraph"
assert len(map2._vertex) == 5, "неверное число вершин в списке _vertex класса LinkedGraph"
map2.add_link(Link(v2, v1))
assert len(map2._links) == 5, "метод add_link() добавил связь Link(v2, v1), хотя уже имеется связь Link(v1, v2)"
path = map2.find_path(v1, v5)
s = sum([x.dist for x in path[1]])
assert s == 3, "неверная суммарная длина маршрута, возможно, некорректно работает объект-свойство dist"
assert issubclass(Station, Vertex) and issubclass(LinkMetro, Link), "класс Station должен наследоваться от класса Vertex, а класс LinkMetro от класса Link"
map2 = LinkedGraph()
v1 = Station("1")
v2 = Station("2")
v3 = Station("3")
v4 = Station("4")
v5 = Station("5")
map2.add_link(LinkMetro(v1, v2, 1))
map2.add_link(LinkMetro(v2, v3, 2))
map2.add_link(LinkMetro(v2, v4, 7))
map2.add_link(LinkMetro(v3, v4, 3))
map2.add_link(LinkMetro(v4, v5, 1))
assert len(map2._links) == 5, "неверное число связей в списке _links класса LinkedGraph"
assert len(map2._vertex) == 5, "неверное число вершин в списке _vertex класса LinkedGraph"
path = map2.find_path(v1, v5)
assert str(path[0]) == '[1, 2, 3, 4, 5]', path[0]
s = sum([x.dist for x in path[1]])
assert s == 7, "неверная суммарная длина маршрута для карты метро"
